In [2]:
with open('prog.asm', 'r') as src:
    source_code = src.readlines()

In [3]:
source_code

['// Computes RAM[1] = 1 + ... + RAM[0]\n',
 '\t@16\n',
 '\tM=1    // i = 1\n',
 '\t@17\n',
 '\tM=0    // sum = 0\n',
 '\n',
 '\t@16    // if i>RAM[0] goto STOP\n',
 '\tD=M\n',
 '\t@0\n',
 '\tD=D-M\n',
 '\t@18\n',
 '\tD;JGT\n',
 '\t@16    // sum += i\n',
 '\tD=M\n',
 '\t@17\n',
 '\tM=D+M\n',
 '\t@16   // i++\n',
 '\tM=M+1\n',
 '\t@4    // goto LOOP\n',
 '\t0;JMP\n',
 '\t@17\n',
 '\tD=M\n',
 '\t@1\n',
 '\tM=D   // RAM[1] = the sum\n',
 '\t@22\n',
 '\t0;JMP\n']

In [4]:
source_code = [line.strip() for line in source_code]

In [5]:
source_code

['// Computes RAM[1] = 1 + ... + RAM[0]',
 '@16',
 'M=1    // i = 1',
 '@17',
 'M=0    // sum = 0',
 '',
 '@16    // if i>RAM[0] goto STOP',
 'D=M',
 '@0',
 'D=D-M',
 '@18',
 'D;JGT',
 '@16    // sum += i',
 'D=M',
 '@17',
 'M=D+M',
 '@16   // i++',
 'M=M+1',
 '@4    // goto LOOP',
 '0;JMP',
 '@17',
 'D=M',
 '@1',
 'M=D   // RAM[1] = the sum',
 '@22',
 '0;JMP']

In [6]:
def remove_comment(string):
    new_str = None
    for i in range(len(string) - 1):
        if string[i] == '/' and string[i+1] == '/':
            new_str = string[:i].strip()
            break
    if new_str is None:
        new_str = string
    return new_str

In [7]:
clean = [remove_comment(line) for line in source_code]
clean = [line for line in clean if (line != '' and line != ' ')]

In [8]:
clean

['@16',
 'M=1',
 '@17',
 'M=0',
 '@16',
 'D=M',
 '@0',
 'D=D-M',
 '@18',
 'D;JGT',
 '@16',
 'D=M',
 '@17',
 'M=D+M',
 '@16',
 'M=M+1',
 '@4',
 '0;JMP',
 '@17',
 'D=M',
 '@1',
 'M=D',
 '@22',
 '0;JMP']

In [9]:
comp_table = {
    '0': '0101010',
    '1': '0111111',
    '-1': '0111010',
    'D': '0001100',
    'A': '0110000',
    'M': '1110000',
    '!D': '0001101',
    '!A': '0110001',
    '!M': '1110001',
    '-D': '0001111',
    '-A': '0110011',
    '-M': '1110011',
    'D+1': '0011111',
    'A+1': '0110111',
    'M+1': '1110111',
    'D-1': '0001110',
    'A-1': '0110010',
    'M-1': '1110010',
    'D+A': '0000010',
    'D+M': '1000010',
    'D-A': '0010011',
    'D-M': '1010011',
    'A-D': '0000111',
    'M-D': '1000111',
    'D&A': '0000000',
    'D&M': '1000000',
    'D|A': '0010101',
    'D|M': '1010101',
}

dest_table = {
    '': '000',
    'M': '001',
    'D': '010',
    'MD': '011',
    'A': '100',
    'AM': '101',
    'AD': '110',
    'AMD': '111',
}

jump_table = {
    '': '000',
    'JGT': '001',
    'JEQ': '010',
    'JGE': '011',
    'JLT': '100',
    'JNE': '101',
    'JLE': '110',
    'JMP': '111',
}

In [32]:
def handle_instruction(inst):
    if inst[0] == '@':
        bin_part = str(bin(int(inst[1:])))[2:]
        if len(bin_part) < 15:
            bin_part = '0'*(15 - len(bin_part)) + bin_part
        return '0' + bin_part + '\n'
    else:
        dest, comp, jump = '', '', ''
        if '=' not in inst and ';' not in inst:
            comp = inst
            print(comp)
        elif ';' not in inst:
            [dest, comp] = inst.split('=')
            print(f'{dest}={comp}')
        elif '=' not in inst:
            [comp, jump] = inst.split(';')
            print(f'{comp};{jump}')
        else:
            [dest, after_eq] = inst.split('=')
            [comp, jump] = after_eq.split(';')
            print(f'{dest}={comp};{jump}')
        final = '111'
        final += comp_table[comp]
        final += dest_table[dest]
        final += jump_table[jump]
        return final + '\n'

In [33]:
clean

['@16',
 'M=1',
 '@17',
 'M=0',
 '@16',
 'D=M',
 '@0',
 'D=D-M',
 '@18',
 'D;JGT',
 '@16',
 'D=M',
 '@17',
 'M=D+M',
 '@16',
 'M=M+1',
 '@4',
 '0;JMP',
 '@17',
 'D=M',
 '@1',
 'M=D',
 '@22',
 '0;JMP']

In [34]:
binary = [handle_instruction(line) for line in clean]

M=1
M=0
D=M
D=D-M
D;JGT
D=M
M=D+M
M=M+1
0;JMP
D=M
M=D
0;JMP


In [35]:
binary

['0000000000010000\n',
 '1110111111001000\n',
 '0000000000010001\n',
 '1110101010001000\n',
 '0000000000010000\n',
 '1111110000010000\n',
 '0000000000000000\n',
 '1111010011010000\n',
 '0000000000010010\n',
 '1110001100000001\n',
 '0000000000010000\n',
 '1111110000010000\n',
 '0000000000010001\n',
 '1111000010001000\n',
 '0000000000010000\n',
 '1111110111001000\n',
 '0000000000000100\n',
 '1110101010000111\n',
 '0000000000010001\n',
 '1111110000010000\n',
 '0000000000000001\n',
 '1110001100001000\n',
 '0000000000010110\n',
 '1110101010000111\n']

In [36]:
with open('prog.hack', 'w') as out:
    out.writelines(binary)

In [1]:
from parse_asm import parse
from symbol_handler import make_symbol_t

instruct_list = parse('./pong.asm')
symbol_t = make_symbol_t(instruct_list)

ponggame.0
math.1
math.0
memory.0
output.4
output.2
output.1
output.0
output.3
output.5
output.6
screen.1
screen.2
screen.0


In [4]:
symbol_t

{'R0': 0,
 'R1': 1,
 'R2': 2,
 'R3': 3,
 'R4': 4,
 'R5': 5,
 'R6': 6,
 'R7': 7,
 'R8': 8,
 'R9': 9,
 'R10': 10,
 'R11': 11,
 'R12': 12,
 'R13': 13,
 'R14': 14,
 'R15': 15,
 'SCREEN': 16384,
 'KBD': 24576,
 'SP': 0,
 'LCL': 1,
 'ARG': 2,
 'THIS': 3,
 'THAT': 4,
 'END_EQ': 19,
 'END_GT': 35,
 'END_LT': 51,
 'RET_ADDRESS_CALL0': 145,
 'ball.new': 145,
 'RET_ADDRESS_CALL1': 163,
 'RET_ADDRESS_CALL2': 333,
 'ball.dispose': 346,
 'RET_ADDRESS_CALL3': 376,
 'ball.show': 387,
 'RET_ADDRESS_CALL4': 418,
 'RET_ADDRESS_CALL5': 441,
 'ball.hide': 452,
 'RET_ADDRESS_CALL6': 480,
 'RET_ADDRESS_CALL7': 503,
 'ball.draw': 514,
 'RET_ADDRESS_CALL8': 588,
 'ball.getleft': 599,
 'ball.getright': 620,
 'ball.setdestination': 652,
 'LOOP_ball.setdestination': 654,
 'RET_ADDRESS_CALL9': 747,
 'RET_ADDRESS_CALL10': 774,
 'RET_ADDRESS_LT0': 798,
 'ball.setdestination$if_true0': 826,
 'RET_ADDRESS_LT1': 886,
 'RET_ADDRESS_LT2': 916,
 'ball.setdestination$if_false0': 930,
 'RET_ADDRESS_LT3': 948,
 'RET_ADDRESS_